In [ ]:
from kubernetes import client, config

# get the capacity and allocatable resources of a node
def get_node_resources(node_name):
    config.load_kube_config()
    api = client.CoreV1Api()
    node = api.read_node(node_name)
    print("node.status.capacity=",node.status.capacity)
    print("node.status.allocatable=",node.status.allocatable)
    
    return node.status.capacity, node.status.allocatable

#convert different memory units to bytes:
def convert_memory_to_bytes(memory_str):
    if 'Gi' in memory_str:
        return int(memory_str.replace('Gi', '')) * 1024 * 1024 * 1024
    elif 'Mi' in memory_str:
        return int(memory_str.replace('Mi', '')) * 1024 * 1024
    elif 'Ki' in memory_str:
        return int(memory_str.replace('Ki', '')) * 1024
    else:
        return int(memory_str)

#convert different memory units to Mega Bytes:
def convert_memory_to_MiB(memory_str):
    if 'Gi' in memory_str:
        return int(memory_str.replace('Gi', '')) * 1024
    elif 'Mi' in memory_str:
        return int(memory_str.replace('Mi', ''))
    else:
        print("Please Check: Memory unit error!")
        return int(memory_str)

# sums up the resource requests and limits for all pods running on a given node
# remove limits []]
def sum_pod_resources_on_node(node_name):
    config.load_kube_config() # Load the kube config to authendicate and interact with the cluster
    api = client.CoreV1Api()

    pod_list = api.list_pod_for_all_namespaces(watch=False) # Get a list of all pods in the cluster
    total_cpu_request, total_memory_request = 0, 0
    total_cpu_limit, total_memory_limit = 0, 0

    for pod in pod_list.items:
        if pod.spec.node_name == node_name:
            for container in pod.spec.containers:
                if container.resources.requests:
                    total_cpu_request += int(container.resources.requests.get('cpu', '0m').replace('m', ''))
                    total_memory_request += int(container.resources.requests.get('memory', '0Mi').replace('Mi', ''))
                if container.resources.limits:
                    total_cpu_limit += int(container.resources.limits.get('cpu', '0m').replace('m', ''))
                    total_memory_limit += convert_memory_to_MiB(container.resources.limits.get('memory', '0Mi'))
    

    return total_cpu_request, total_memory_request, total_cpu_limit, total_memory_limit

# vertically scale the resources of a specific pod
def vertical_scale_pod_resources2(pod_name, namespace, new_cpu_request, new_cpu_limit, new_memory_request, new_memory_limit):
    # Load the kube config
    config.load_kube_config()
    # Create an CoreV1Api instance
    api = client.CoreV1Api()

    # Fetch the existing pod
    pod = api.read_namespaced_pod(pod_name, namespace)
    container = pod.spec.containers[0] # Get the first container in the pod; change to differnet containers index if needed

    # Get the node name from the pod spec
    node_name = pod.spec.node_name

    node_capacity, node_allocatable = get_node_resources(node_name)
    node_cpu_allocatable_millicores = int(node_allocatable['cpu'].replace('m', ''))*1000 # convert to millicores
    node_memory_allocatable_mebibytes = int(node_allocatable['memory'].replace('Ki', ''))/1024  # convert to MiB
    
    total_cpu_request, total_memory_request, total_cpu_limit, total_memory_limit = sum_pod_resources_on_node(node_name)

    new_cpu_request_millicores = int(new_cpu_request.replace('m', ''))
    new_cpu_limit_millicores = int(new_cpu_limit.replace('m', ''))
    new_memory_request_mebibytes = int(new_memory_request.replace('Mi', ''))
    new_memory_limit_mebibytes = int(new_memory_limit.replace('Mi', ''))

    # Check whether the node can accommodate the new requests and limits
    if (total_cpu_request + new_cpu_request_millicores <= node_cpu_allocatable_millicores and
        total_memory_request + new_memory_request_mebibytes <= node_memory_allocatable_mebibytes and
        total_cpu_limit + new_cpu_limit_millicores <= node_cpu_allocatable_millicores and
        total_memory_limit + new_memory_limit_mebibytes <= node_memory_allocatable_mebibytes):

        if not container.resources:
            container.resources = client.V1ResourceRequirements()

        container.resources.requests = {
            "cpu": new_cpu_request,
            "memory": new_memory_request
        }

        container.resources.limits = {
            "cpu": new_cpu_limit,
            "memory": new_memory_limit
        }

        # Apply the updated pod spec
        api.patch_namespaced_pod(pod_name, namespace, pod)

        print(f"Successfully scaled the resources for pod {pod_name}")
    else:
        print("Insufficient node resources for the requested scaling operation.")

In [ ]:
# vertically scale the resources of pods from a deployment
def vertical_scale_pod_resources(deployment_name, namespace, new_cpu_request, new_cpu_limit, new_memory_request, new_memory_limit, node_name):
    # Load the kube config
    config.load_kube_config()
    # Create an AppsV1Api instance
    api = client.AppsV1Api()

    # Fetch the existing deployment
    deployment = api.read_namespaced_deployment(deployment_name, namespace)
    container = deployment.spec.template.spec.containers[0]

    node_capacity, node_allocatable = get_node_resources(node_name)
    node_cpu_allocatable_millicores = int(node_allocatable['cpu'].replace('m', ''))*1000 # convert to millicores
    node_memory_allocatable_mebibytes = int(node_allocatable['memory'].replace('Ki', ''))/1024  # convert to MiB
    
    total_cpu_request, total_memory_request, total_cpu_limit, total_memory_limit = sum_pod_resources_on_node(node_name)

    new_cpu_request_millicores = int(new_cpu_request.replace('m', ''))
    new_cpu_limit_millicores = int(new_cpu_limit.replace('m', ''))
    new_memory_request_mebibytes = int(new_memory_request.replace('Mi', ''))
    new_memory_limit_mebibytes = int(new_memory_limit.replace('Mi', ''))

    # Check whether the node can accommodate the new requests and limits
    if (total_cpu_request + new_cpu_request_millicores <= node_cpu_allocatable_millicores and
        total_memory_request + new_memory_request_mebibytes <= node_memory_allocatable_mebibytes and
        total_cpu_limit + new_cpu_limit_millicores <= node_cpu_allocatable_millicores and
        total_memory_limit + new_memory_limit_mebibytes <= node_memory_allocatable_mebibytes):

        if not container.resources:
            container.resources = client.V1ResourceRequirements()

        container.resources.requests = {
            "cpu": new_cpu_request,
            "memory": new_memory_request
        }

        container.resources.limits = {
            "cpu": new_cpu_limit,
            "memory": new_memory_limit
        }

        # Apply the updated deployment spec
        api.patch_namespaced_deployment(deployment_name, namespace, deployment)

        print(f"Successfully scaled the resources for deployment {deployment_name}")
    else:
        print("Insufficient node resources for the requested scaling operation.")
        ######################### Conditio  Check, why failed to scale ##########################################
        
        node_cpu_allocatable_millicores = int(node_allocatable['cpu'].replace('m', ''))*1000 # convert to millicores
        node_memory_allocatable_mebibytes = int(node_allocatable['memory'].replace('Ki', ''))/1024  # convert to MiB

        # Calculate the new total values after scaling
        new_total_cpu_request = total_cpu_request + new_cpu_request_millicores
        new_total_memory_request = total_memory_request + new_memory_request_mebibytes
        new_total_cpu_limit = total_cpu_limit + new_cpu_limit_millicores
        new_total_memory_limit = total_memory_limit + new_memory_limit_mebibytes

        # Printout for debugging
        print(f"New total CPU request: {new_total_cpu_request} (Available: {node_cpu_allocatable_millicores})")
        print(f"New total Memory request: {new_total_memory_request} (Available: {node_memory_allocatable_mebibytes})")
        print(f"New total CPU limit: {new_total_cpu_limit} (Available: {node_cpu_allocatable_millicores})")
        print(f"New total Memory limit: {new_total_memory_limit} (Available: {node_memory_allocatable_mebibytes})")

        # Check conditions
        cpu_request_condition = new_total_cpu_request <= node_cpu_allocatable_millicores
        memory_request_condition = new_total_memory_request <= node_memory_allocatable_mebibytes
        cpu_limit_condition = new_total_cpu_limit <= node_cpu_allocatable_millicores
        memory_limit_condition = new_total_memory_limit <= node_memory_allocatable_mebibytes

        # Printout for debugging which condition fails
        if not cpu_request_condition:
            print("Condition failed: CPU Request exceeds available allocatable CPU.")
        if not memory_request_condition:
            print("Condition failed: Memory Request exceeds available allocatable memory.")
        if not cpu_limit_condition:
            print("Condition failed: CPU Limit exceeds available allocatable CPU.")
        if not memory_limit_condition:
            print("Condition failed: Memory Limit exceeds available allocatable memory.")

        # Final condition check
        if cpu_request_condition and memory_request_condition and cpu_limit_condition and memory_limit_condition:
            # Proceed with the scaling operation
            print("All conditions met. Proceeding with scaling.")
        else:
            print("Scaling operation aborted. One or more conditions not met.")

In [ ]:
# Test usage by scaling pod resources via deployment
deployment_name = "nginx-deployment"
namespace = "nginx-app"
node_name = "microk8snode1"

new_cpu_request = "20m" # 20 millicores
new_cpu_limit = "40m"
new_memory_request = "16Mi" # 16 Mebibytes
new_memory_limit = "30Mi"

vertical_scale_pod_resources(deployment_name, namespace, new_cpu_request, new_cpu_limit, new_memory_request, new_memory_limit, node_name)